In [1]:
import osmnx as ox
import geopandas as gpd
import pandas as pd
import requests
import time
from shapely.geometry import box

In [14]:
# --- 1. Baixar o polígono da Cidade Tiradentes ---
bairro = ox.geocode_to_gdf("Itaquera, São Paulo, Brasil")

# --- 2. Criar a grade (grid) sobre o bairro ---
def criar_grid(bairro, tamanho=0.005):  # Aproximadamente 200m
    minx, miny, maxx, maxy = bairro.total_bounds
    grid = []
    x = minx
    while x < maxx:
        y = miny
        while y < maxy:
            grid.append(box(x, y, x + tamanho, y + tamanho))
            y += tamanho
        x += tamanho
    grid_gdf = gpd.GeoDataFrame(geometry=grid, crs=bairro.crs)
    grid_gdf = gpd.overlay(grid_gdf, bairro, how='intersection')
    return grid_gdf

grid = criar_grid(bairro)


# --- 3. Corrigir projeção para metros, calcular centroide ---
grid = grid.to_crs(epsg=3857)  # Web Mercator
grid["centroide"] = grid.geometry.centroid

# --- 4. Criar colunas separadas de latitude/longitude
centroides = grid["centroide"].to_crs(epsg=4326)  # voltar para lat/lon
grid["centroide_lon"] = centroides.geometry.x
grid["centroide_lat"] = centroides.geometry.y

# --- 6. Definir hospital e API ---
API_KEY = "5b3ce3597851110001cf624824d680a2a89c4dc48933794aaca7b81d"  # Troque aqui pela sua chave
hospital_coords = [-46.4519756, -23.5351716] 

def calcular_distancia_km(origem, destino, api_key):
    url = "https://api.openrouteservice.org/v2/directions/driving-car/json"
    headers = {
        "Authorization": api_key,
        "Content-Type": "application/json"
    }
    body = {
        "coordinates": [origem, destino]
    }
    try:
        response = requests.post(url, json=body, headers=headers)
        if response.status_code == 200:
            dados = response.json()
            distancia_metros = dados['routes'][0]['summary']['distance']
            distancia_km = distancia_metros / 1000
            return round(distancia_km, 2)
        else:
            print("Erro na API:", response.text)
            return None
    except Exception as e:
        print(f"Erro: {e}")
        return None

# --- 7. Calcular a distância para cada grid ---
distancias_km = []
for i, row in grid.iterrows():
    origem = [row["centroide_lon"], row["centroide_lat"]]
    distancia = calcular_distancia_km(origem, hospital_coords, API_KEY)
    distancias_km.append(distancia)
    time.sleep(1)  # evita limite da API gratuita

grid["distancia_km"] = distancias_km
grid.to_csv(f"estimativa_itaquera.csv", index=False)

In [18]:
grid

,bbox_west,bbox_south,bbox_east,bbox_north,place_id,osm_type,osm_id,lat,lon,class,...,place_rank,importance,addresstype,name,display_name,geometry,centroide,centroide_lon,centroide_lat,distancia_km
0,-46.477552,-23.556978,-46.429414,-23.513474,8010704,relation,3049502,-23.53608,-46.45551,boundary,...,18,0.402573,suburb,Itaquera,"Itaquera, São Paulo, Região Imediata de São Pa...","POLYGON ((-5173300.825 -2698301.772, -5173300....",POINT (-5173559.063 -2698421.165),-46.474872,-23.547961,3.93
1,-46.477552,-23.556978,-46.429414,-23.513474,8010704,relation,3049502,-23.53608,-46.45551,boundary,...,18,0.402573,suburb,Itaquera,"Itaquera, São Paulo, Região Imediata de São Pa...","POLYGON ((-5173300.825 -2697694.63, -5173300.8...",POINT (-5173488.105 -2698065.107),-46.474234,-23.545029,4.84
2,-46.477552,-23.556978,-46.429414,-23.513474,8010704,relation,3049502,-23.53608,-46.45551,boundary,...,18,0.402573,suburb,Itaquera,"Itaquera, São Paulo, Região Imediata de São Pa...","POLYGON ((-5173300.825 -2697087.511, -5173300....",POINT (-5173405.757 -2697322.729),-46.473495,-23.538915,3.04
3,-46.477552,-23.556978,-46.429414,-23.513474,8010704,relation,3049502,-23.53608,-46.45551,boundary,...,18,0.402573,suburb,Itaquera,"Itaquera, São Paulo, Região Imediata de São Pa...","POLYGON ((-5173300.825 -2696480.415, -5173300....",POINT (-5173385.464 -2696838.732),-46.473312,-23.534929,3.34
4,-46.477552,-23.556978,-46.429414,-23.513474,8010704,relation,3049502,-23.53608,-46.45551,boundary,...,18,0.402573,suburb,Itaquera,"Itaquera, São Paulo, Região Imediata de São Pa...","POLYGON ((-5173300.825 -2695873.343, -5173300....",POINT (-5173384.225 -2696144.68),-46.473301,-23.529213,4.13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,-46.477552,-23.556978,-46.429414,-23.513474,8010704,relation,3049502,-23.53608,-46.45551,boundary,...,18,0.402573,suburb,Itaquera,"Itaquera, São Paulo, Região Imediata de São Pa...","POLYGON ((-5169404.642 -2696480.415, -5169404....",POINT (-5169126.344 -2696176.879),-46.435052,-23.529478,3.38
67,-46.477552,-23.556978,-46.429414,-23.513474,8010704,relation,3049502,-23.53608,-46.45551,boundary,...,18,0.402573,suburb,Itaquera,"Itaquera, São Paulo, Região Imediata de São Pa...","POLYGON ((-5168848.045 -2695873.343, -5169404....",POINT (-5169117.535 -2695766.527),-46.434973,-23.526098,3.35
68,-46.477552,-23.556978,-46.429414,-23.513474,8010704,relation,3049502,-23.53608,-46.45551,boundary,...,18,0.402573,suburb,Itaquera,"Itaquera, São Paulo, Região Imediata de São Pa...","POLYGON ((-5168848.045 -2696480.415, -5168546....",POINT (-5168694.051 -2696600.105),-46.431169,-23.532964,3.60
69,-46.477552,-23.556978,-46.429414,-23.513474,8010704,relation,3049502,-23.53608,-46.45551,boundary,...,18,0.402573,suburb,Itaquera,"Itaquera, São Paulo, Região Imediata de São Pa...","POLYGON ((-5168848.045 -2696480.415, -5168848....",POINT (-5168739.25 -2696245.348),-46.431575,-23.530042,3.95


In [19]:
%pip install folium
import folium
from shapely.geometry import mapping

# Converter o bairro para EPSG:4326 (lat/lon) para exibição no mapa
bairro_wgs84 = bairro.to_crs(epsg=4326)
grid_wgs84 = grid.to_crs(epsg=4326)

# Criar mapa centralizado no bairro
centroide_bairro = bairro_wgs84.geometry.centroid.iloc[0]
m = folium.Map(location=[centroide_bairro.y, centroide_bairro.x], zoom_start=14)

# Adicionar polígono do bairro
folium.GeoJson(bairro_wgs84.geometry.__geo_interface__,
               name="Bairro - Cidade Tiradentes",
               style_function=lambda x: {"color": "black", "fillOpacity": 0.1}
).add_to(m)

# Adicionar os grids
for _, row in grid_wgs84.iterrows():
    folium.GeoJson(mapping(row.geometry),
                   style_function=lambda x: {"color": "blue", "weight": 1, "fillOpacity": 0}
    ).add_to(m)

# Adicionar ponto do hospital
folium.Marker(
    location=[-23.5351716, -46.4519756] ,
    popup="Hospital Itaquera",
    icon=folium.Icon(color='red', icon='plus-sign')
).add_to(m)

# Mostrar mapa
m


[notice] A new release of pip is available: 23.2.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip
C:\Users\joth1\AppData\Local\Temp\ipykernel_7664\3008177741.py:10: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroide_bairro = bairro_wgs84.geometry.centroid.iloc[0]


Note: you may need to restart the kernel to use updated packages.


In [20]:

from geopy.geocoders import Nominatim
import time

# Inicializando o geolocalizador
geolocator = Nominatim(user_agent="meu_aplicativo")

def buscar_samus_em_sao_paulo(bairro, nome_variavel):
    print(f"Buscando Hospital em {bairro}...")
    try:
        localizacao = geolocator.geocode(bairro)
        if localizacao:
            # Montando o retorno no formato desejado
            print(f"{nome_variavel} = ({localizacao.latitude}, {localizacao.longitude})")
            return (localizacao.latitude, localizacao.longitude)
        else:
            print(f"Não encontrado no bairro: {bairro}")
            return None
    except Exception as e:
        print(f"Erro ao buscar em {bairro}: {e}")
        time.sleep(1)
        return None

# Exemplo de uso}
coordenadas = buscar_samus_em_sao_paulo('VACANGA,R,300', 'arne_')

Buscando Hospital em VACANGA,R,300...
arne_ = (-23.5605378, -46.5315852)
